# GAN-BERT (in Pytorch and compatible with HuggingFace)

This is a Pytorch (+ **Huggingface** transformers) implementation of the GAN-BERT model from https://github.com/crux82/ganbert. While the original GAN-BERT was an extension of BERT, this implementation can be adapted to several architectures, ranging from Roberta to Albert!

**NOTE**: given that this implementation is different from the original one in Tensorflow, some results can be slighty different.


Let's GO!

Required Imports.

In [12]:
import time
start_time = time.time()
!pip install transformers==4.3.2
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install sentencepiece

##Set random values
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed_val)

  Using cached transformers-4.3.2-py3-none-any.whl.metadata (36 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached transformers-4.3.2-py3-none-any.whl (1.8 MB)
Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [13]:
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


### Input Parameters


In [14]:
from transformers import AutoTokenizer
#--------------------------------
#  Transformer parameters
#--------------------------------
max_seq_length = 64
batch_size = 64

#--------------------------------
#  GAN-BERT specific parameters
#--------------------------------
# number of hidden layers in the generator,
# each of the size of the output space
num_hidden_layers_g = 1;
# number of hidden layers in the discriminator,
# each of the size of the input space
num_hidden_layers_d = 1;
# size of the generator's input noisy vectors
noise_size = 100
# dropout to be applied to discriminator's input vectors
out_dropout_rate = 0.1

# Replicate labeled data to balance poorly represented datasets,
# e.g., less than 1% of labeled material
apply_balance = True

#--------------------------------
#  Optimization parameters
#--------------------------------
learning_rate_discriminator = 5e-5
learning_rate_generator = 5e-5
epsilon = 1e-8
num_train_epochs = 10
multi_gpu = True
# Scheduler
apply_scheduler = False
warmup_proportion = 0.1
# Print
print_each_n_step = 10

#--------------------------------
#  Adopted Tranformer model
#--------------------------------
# Since this version is compatible with Huggingface transformers, you can uncomment
# (or add) transformer models compatible with GAN

model_name = "bert-base-cased"
#model_name = "bert-base-uncased"
#model_name = "roberta-base"
#model_name = "albert-base-v2" TESTAR
#model_name = "xlm-roberta-base"
#model_name = "amazon/bort"

#--------------------------------
#  Retrieve the TREC QC Dataset
#--------------------------------
! git clone https://github.com/mauriciokonrath/ganbert.git

#  NOTE: in this setting 50 classes are involved
labeled_file = "./ganbert/data/standardized_test_withoutSub.tsv"
unlabeled_file = "./ganbert/data/unlabeled.tsv"
test_filename = "./ganbert/data/standardized_labeled_withoutSub.tsv"


label_list = ["UNK_UNK","NUM_num", "DESC_desc", "HUM_hum",
              "ENTY_enty", "LOC_loc"]

fatal: destination path 'ganbert' already exists and is not an empty directory.


Load the Tranformer Model

In [15]:
transformer = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/model.safetenso

Function required to load the dataset

In [16]:
def get_qc_examples(input_file):
  """Creates examples for the training and dev sets."""
  examples = []

  with open(input_file, 'r') as f:
      contents = f.read()
      file_as_list = contents.splitlines()
      for line in file_as_list[1:]:
          split = line.split(" ")
          question = ' '.join(split[1:])

          text_a = question
          inn_split = split[0].split(":")
          label = inn_split[0] + "_" + inn_split[1]
          examples.append((text_a, label))
      f.close()

  return examples

**Load** the input QC dataset (fine-grained)

In [17]:

#Load the examples
labeled_examples = get_qc_examples(labeled_file)
unlabeled_examples = get_qc_examples(unlabeled_file)
test_examples = get_qc_examples(test_filename)


Functions required to convert examples into Dataloader

In [18]:
def generate_data_loader(input_examples, label_masks, label_map, do_shuffle = False, balance_label_examples = False):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are
  to be considered NOT labeled.
  '''
  examples = []

  # Count the percentage of labeled examples
  num_labeled_examples = 0
  for label_mask in label_masks:
    if label_mask:
      num_labeled_examples += 1
  label_mask_rate = num_labeled_examples/len(input_examples)

  # if required it applies the balance
  for index, ex in enumerate(input_examples):
    if label_mask_rate == 1 or not balance_label_examples:
      examples.append((ex, label_masks[index]))
    else:
      # IT SIMULATE A LABELED EXAMPLE
      if label_masks[index]:
        balance = int(1/label_mask_rate)
        balance = int(math.log(balance,2))
        if balance < 1:
          balance = 1
        for b in range(0, int(balance)):
          examples.append((ex, label_masks[index]))
      else:
        examples.append((ex, label_masks[index]))

  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []
  label_mask_array = []
  label_id_array = []

  # Tokenization
  for (text, label_mask) in examples:
    encoded_sent = tokenizer.encode(text[0], add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
    label_id_array.append(label_map[text[1]])
    label_mask_array.append(label_mask)

  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids)
  input_mask_array = torch.tensor(input_mask_array)
  label_id_array = torch.tensor(label_id_array, dtype=torch.long)
  label_mask_array = torch.tensor(label_mask_array)

  # Building the TensorDataset
  dataset = TensorDataset(input_ids, input_mask_array, label_id_array, label_mask_array)

  if do_shuffle:
    sampler = RandomSampler
  else:
    sampler = SequentialSampler

  # Building the DataLoader
  return DataLoader(
              dataset,  # The training samples.
              sampler = sampler(dataset),
              batch_size = batch_size) # Trains with this batch size.

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


Convert the input examples into DataLoader

In [19]:

label_map = {}
for (i, label) in enumerate(label_list):
  label_map[label] = i
#------------------------------
#   Load the train dataset
#------------------------------
train_examples = labeled_examples
#The labeled (train) dataset is assigned with a mask set to True
train_label_masks = np.ones(len(labeled_examples), dtype=bool)
#If unlabel examples are available
if unlabeled_examples:
  train_examples = train_examples + unlabeled_examples
  #The unlabeled (train) dataset is assigned with a mask set to False
  tmp_masks = np.zeros(len(unlabeled_examples), dtype=bool)
  train_label_masks = np.concatenate([train_label_masks,tmp_masks])

train_dataloader = generate_data_loader(train_examples, train_label_masks, label_map, do_shuffle = True, balance_label_examples = apply_balance)

#------------------------------
#   Load the test dataset
#------------------------------
#The labeled (test) dataset is assigned with a mask set to True
test_label_masks = np.ones(len(test_examples), dtype=bool)

test_dataloader = generate_data_loader(test_examples, test_label_masks, label_map, do_shuffle = False, balance_label_examples = False)


<ipython-input-18-16c38d5f7491>:54: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  label_mask_array = torch.tensor(label_mask_array)


We define the Generator and Discriminator as discussed in https://www.aclweb.org/anthology/2020.acl-main.191/

In [20]:

#------------------------------
#   The Generator as in
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Generator(nn.Module):
    def __init__(self, noise_size=100, output_size=512, hidden_sizes=[512], dropout_rate=0.1):
        super(Generator, self).__init__()
        layers = []
        hidden_sizes = [noise_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        layers.append(nn.Linear(hidden_sizes[-1],output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, noise):
        output_rep = self.layers(noise)
        return output_rep

#------------------------------
#   The Discriminator
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Discriminator(nn.Module):
    def __init__(self, input_size=512, hidden_sizes=[512], num_labels=2, dropout_rate=0.1):
        super(Discriminator, self).__init__()
        self.input_dropout = nn.Dropout(p=dropout_rate)
        layers = []
        hidden_sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        self.layers = nn.Sequential(*layers) #per il flatten
        self.logit = nn.Linear(hidden_sizes[-1],num_labels+1) # +1 for the probability of this sample being fake/real.
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_rep):
        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)
        logits = self.logit(last_rep)
        probs = self.softmax(logits)
        return last_rep, logits, probs

We instantiate the Discriminator and Generator

In [21]:

# The config file is required to get the dimension of the vector produced by
# the underlying transformer
config = AutoConfig.from_pretrained(model_name)
hidden_size = int(config.hidden_size)
# Define the number and width of hidden layers
hidden_levels_g = [hidden_size for i in range(0, num_hidden_layers_g)]
hidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]

#-------------------------------------------------
#   Instantiate the Generator and Discriminator
#-------------------------------------------------
generator = Generator(noise_size=noise_size, output_size=hidden_size, hidden_sizes=hidden_levels_g, dropout_rate=out_dropout_rate)
discriminator = Discriminator(input_size=hidden_size, hidden_sizes=hidden_levels_d, num_labels=len(label_list), dropout_rate=out_dropout_rate)

# Put everything in the GPU if available
if torch.cuda.is_available():
  generator.cuda()
  discriminator.cuda()
  transformer.cuda()
  if multi_gpu:
    transformer = torch.nn.DataParallel(transformer)

# print(config)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



Let's go with the training procedure

In [22]:
import time
import torch
import torch.nn.functional as F
import numpy as np

# Função para calcular Recall, Precision e F1-Score manualmente para múltiplas classes
def calculate_recall_precision_f1_multiclass(y_true, y_pred):
    # Classes únicas
    classes = np.unique(y_true)

    recalls = []
    precisions = []
    f1_scores = []

    for cls in classes:
        # True Positives, False Positives, False Negatives para a classe atual
        tp = np.sum((y_true == cls) & (y_pred == cls))
        fp = np.sum((y_true != cls) & (y_pred == cls))
        fn = np.sum((y_true == cls) & (y_pred != cls))

        # Recall
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        recalls.append(recall)

        # Precision
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        precisions.append(precision)

        # F1 Score
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)

    # Média das métricas por classe
    avg_recall = np.mean(recalls)
    avg_precision = np.mean(precisions)
    avg_f1 = np.mean(f1_scores)

    return avg_recall, avg_precision, avg_f1

# Função auxiliar para formatar o tempo
def format_time(elapsed):
    return str(round(elapsed, 2)) + "s"

training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# Optimizers
transformer_vars = [i for i in transformer.parameters()]
d_vars = transformer_vars + [v for v in discriminator.parameters()]
g_vars = [v for v in generator.parameters()]

dis_optimizer = torch.optim.AdamW(d_vars, lr=learning_rate_discriminator)
gen_optimizer = torch.optim.AdamW(g_vars, lr=learning_rate_generator)

# Scheduler
if apply_scheduler:
    num_train_examples = len(train_examples)
    num_train_steps = int(num_train_examples / batch_size * num_train_epochs)
    num_warmup_steps = int(num_train_steps * warmup_proportion)

    scheduler_d = get_constant_schedule_with_warmup(dis_optimizer, num_warmup_steps=num_warmup_steps)
    scheduler_g = get_constant_schedule_with_warmup(gen_optimizer, num_warmup_steps=num_warmup_steps)

# Treinamento do modelo
for epoch_i in range(0, num_train_epochs):
    print("")
    print(f"======== Epoch {epoch_i + 1} / {num_train_epochs} ========")
    print("Training...")

    t0 = time.time()
    tr_g_loss = 0
    tr_d_loss = 0

    transformer.train()
    generator.train()
    discriminator.train()

    for step, batch in enumerate(train_dataloader):
        if step % print_each_n_step == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f"  Batch {step:,}  of  {len(train_dataloader):,}.    Elapsed: {elapsed}.")

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_label_mask = batch[3].to(device)

        real_batch_size = b_input_ids.shape[0]

        model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
        hidden_states = model_outputs[-1]

        noise = torch.zeros(real_batch_size, noise_size, device=device).uniform_(0, 1)
        gen_rep = generator(noise)

        disciminator_input = torch.cat([hidden_states, gen_rep], dim=0)
        features, logits, probs = discriminator(disciminator_input)

        features_list = torch.split(features, real_batch_size)
        D_real_features, D_fake_features = features_list

        logits_list = torch.split(logits, real_batch_size)
        D_real_logits, D_fake_logits = logits_list

        probs_list = torch.split(probs, real_batch_size)
        D_real_probs, D_fake_probs = probs_list

        # Generator loss
        g_loss_d = -torch.mean(torch.log(1 - D_fake_probs[:, -1] + epsilon))
        g_feat_reg = torch.mean(torch.pow(torch.mean(D_real_features, dim=0) - torch.mean(D_fake_features, dim=0), 2))
        g_loss = g_loss_d + g_feat_reg

        # Discriminator loss
        logits = D_real_logits[:, :-1]
        log_probs = F.log_softmax(logits, dim=-1)
        label2one_hot = F.one_hot(b_labels, len(label_list))
        per_example_loss = -torch.sum(label2one_hot * log_probs, dim=-1)
        per_example_loss = torch.masked_select(per_example_loss, b_label_mask.to(device))
        labeled_example_count = per_example_loss.numel()

        if labeled_example_count == 0:
            D_L_Supervised = 0
        else:
            D_L_Supervised = torch.sum(per_example_loss) / labeled_example_count

        D_L_unsupervised1U = -torch.mean(torch.log(1 - D_real_probs[:, -1] + epsilon))
        D_L_unsupervised2U = -torch.mean(torch.log(D_fake_probs[:, -1] + epsilon))
        d_loss = D_L_Supervised + D_L_unsupervised1U + D_L_unsupervised2U

        gen_optimizer.zero_grad()
        dis_optimizer.zero_grad()

        g_loss.backward(retain_graph=True)
        d_loss.backward()

        gen_optimizer.step()
        dis_optimizer.step()

        tr_g_loss += g_loss.item()
        tr_d_loss += d_loss.item()

        if apply_scheduler:
            scheduler_d.step()
            scheduler_g.step()

    avg_train_loss_g = tr_g_loss / len(train_dataloader)
    avg_train_loss_d = tr_d_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)

    print(f"  Average training loss generator: {avg_train_loss_g:.3f}")
    print(f"  Average training loss discriminator: {avg_train_loss_d:.3f}")
    print(f"  Training epoch took: {training_time}")

    # Avaliação por época para calcular a acurácia
    print("\nRunning Test...")

    all_preds = []
    all_labels_ids = []
    total_test_loss = 0

    transformer.eval()
    discriminator.eval()
    generator.eval()

    for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
            hidden_states = model_outputs[-1]
            _, logits, probs = discriminator(hidden_states)
            filtered_logits = logits[:, :-1]
            total_test_loss += F.cross_entropy(filtered_logits, b_labels, ignore_index=-1)

            _, preds = torch.max(filtered_logits, 1)
            all_preds += preds.detach().cpu()
            all_labels_ids += b_labels.detach().cpu()

    # Cálculo da acurácia para cada rodada
    all_preds = torch.stack(all_preds).numpy()
    all_labels_ids = torch.stack(all_labels_ids).numpy()
    test_accuracy = np.sum(all_preds == all_labels_ids) / len(all_preds)

    print(f"  Accuracy: {test_accuracy:.3f}")
    avg_test_loss = total_test_loss / len(test_dataloader)
    avg_test_loss = avg_test_loss.item()
    test_time = format_time(time.time() - t0)

    print(f"  Test Loss: {avg_test_loss:.3f}")
    print(f"  Test took: {test_time}")

    training_stats.append({
        'epoch': epoch_i + 1,
        'Training Loss generator': avg_train_loss_g,
        'Training Loss discriminator': avg_train_loss_d,
        'Valid. Loss': avg_test_loss,
        'Valid. Accur.': test_accuracy,
        'Training Time': training_time,
        'Test Time': test_time
    })

# Avaliação do modelo no final do treinamento
print("\nFinal Evaluation...")

recall, precision, f1 = calculate_recall_precision_f1_multiclass(all_labels_ids, all_preds)

print(f"Final Recall: {recall:.3f}")
print(f"Final Precision: {precision:.3f}")
print(f"Final F1 Score: {f1:.3f}")



======== Epoch 1 / 10 ========
Training...
  Batch 10  of  104.    Elapsed: 9.29s.
  Batch 20  of  104.    Elapsed: 18.67s.
  Batch 30  of  104.    Elapsed: 28.2s.
  Batch 40  of  104.    Elapsed: 37.83s.
  Batch 50  of  104.    Elapsed: 47.35s.
  Batch 60  of  104.    Elapsed: 56.74s.
  Batch 70  of  104.    Elapsed: 66.04s.
  Batch 80  of  104.    Elapsed: 75.28s.
  Batch 90  of  104.    Elapsed: 84.5s.
  Batch 100  of  104.    Elapsed: 93.75s.
  Average training loss generator: 0.670
  Average training loss discriminator: 1.650
  Training epoch took: 97.29s

Running Test...
  Accuracy: 0.860
  Test Loss: 0.582
  Test took: 97.46s

======== Epoch 2 / 10 ========
Training...
  Batch 10  of  104.    Elapsed: 9.33s.
  Batch 20  of  104.    Elapsed: 18.71s.
  Batch 30  of  104.    Elapsed: 28.11s.
  Batch 40  of  104.    Elapsed: 37.51s.
  Batch 50  of  104.    Elapsed: 46.87s.
  Batch 60  of  104.    Elapsed: 56.2s.
  Batch 70  of  104.    Elapsed: 65.51s.
  Batch 80  of  104.    Elaps

In [23]:
for stat in training_stats:
  print(stat)

print("\nTraining complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

resultado = sum(range(10**6))

end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time:.4f} segundos")

{'epoch': 1, 'Training Loss generator': 0.6696677081860029, 'Training Loss discriminator': 1.6498607110518675, 'Valid. Loss': 0.5822218060493469, 'Valid. Accur.': 0.86, 'Training Time': '97.29s', 'Test Time': '97.46s'}
{'epoch': 2, 'Training Loss generator': 0.726093362730283, 'Training Loss discriminator': 0.7960128927460084, 'Valid. Loss': 0.4173244833946228, 'Valid. Accur.': 0.92, 'Training Time': '97.0s', 'Test Time': '97.16s'}
{'epoch': 3, 'Training Loss generator': 0.707857839190043, 'Training Loss discriminator': 0.7223200528667524, 'Valid. Loss': 0.3639635443687439, 'Valid. Accur.': 0.92, 'Training Time': '96.9s', 'Test Time': '97.06s'}
{'epoch': 4, 'Training Loss generator': 0.7001083252521662, 'Training Loss discriminator': 0.7128822494011658, 'Valid. Loss': 0.4301259219646454, 'Valid. Accur.': 0.92, 'Training Time': '96.95s', 'Test Time': '97.11s'}
{'epoch': 5, 'Training Loss generator': 0.698460272871531, 'Training Loss discriminator': 0.708876904959862, 'Valid. Loss': 0.44

In [24]:
#ADICIONADO
# Salvar os modelos
torch.save(generator.state_dict(), 'generator.pt')
torch.save(discriminator.state_dict(), 'discriminator.pt')
torch.save(transformer.state_dict(), 'transformer.pt')

In [25]:
# Carregar os modelos
generator.load_state_dict(torch.load('generator.pt'))
discriminator.load_state_dict(torch.load('discriminator.pt'))
transformer.load_state_dict(torch.load('transformer.pt'))


<ipython-input-25-a3c231b017ae>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load('generator.pt'))
<ipython-input-25-a3c231b017ae>:3: Fut

<All keys matched successfully>

In [26]:
#ADICIONADO
from transformers import AutoTokenizer, AutoModel, AutoConfig

# Nome do modelo usado durante o treinamento
model_name = "bert-base-cased"

# Carregar tokenizer e configuração
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
hidden_size = int(config.hidden_size)


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/vocab.txt
loading file tokenize

In [27]:
#ADICIONADO
def preprocess(text, max_seq_length=64):
    encoded_sent = tokenizer.encode(text, add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
    input_ids = torch.tensor([encoded_sent])
    attention_mask = torch.tensor([[int(token_id > 0) for token_id in encoded_sent]])
    return input_ids, attention_mask


In [28]:
#ADICIONADO
def classify_question(text):
    transformer.eval()
    discriminator.eval()

    # Pré-processar a nova pergunta
    input_ids, attention_mask = preprocess(text)

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    with torch.no_grad():
        # Passar pela transformer
        model_outputs = transformer(input_ids, attention_mask=attention_mask)
        hidden_states = model_outputs[-1]

        # Passar pelo discriminator
        _, logits, probs = discriminator(hidden_states)

        # Filtrar os logits
        filtered_logits = logits[:, :-1]

        # Obter a predição
        pred = torch.argmax(filtered_logits, dim=1).item()

    return label_list[pred]

# Exemplo de uso
new_question = "Who is the current president of the United States?"
predicted_label = classify_question(new_question)
print(f"Predicted Label: {predicted_label}")


Predicted Label: HUM_hum


In [29]:
'''import nltk
from nltk.tokenize import sent_tokenize

# Baixar o recurso de pontuação da NLTK (apenas na primeira vez)
nltk.download("punkt")
# Texto de entrada
text = "I’ve been asked to review a recent study on glyphosate’s cytotoxicity in rats. Given your expertise in the field, I believe it would be beneficial for both of you to act as peer reviewers. Once you’ve collated your comments, I’ll submit the final review on our behalf."

# Dividir o texto em sentenças
sentences = sent_tokenize(text)
# Classificar cada sentença individualmente
for sentence in sentences:
    predicted_label = classify_question(sentence)
    print(f"Sentence: {sentence}")
    print(f"Predicted Label: {predicted_label}\n")'''



'import nltk\nfrom nltk.tokenize import sent_tokenize\n\n# Baixar o recurso de pontuação da NLTK (apenas na primeira vez)\nnltk.download("punkt")\n# Texto de entrada\ntext = "I’ve been asked to review a recent study on glyphosate’s cytotoxicity in rats. Given your expertise in the field, I believe it would be beneficial for both of you to act as peer reviewers. Once you’ve collated your comments, I’ll submit the final review on our behalf."\n\n# Dividir o texto em sentenças\nsentences = sent_tokenize(text)\n# Classificar cada sentença individualmente\nfor sentence in sentences:\n    predicted_label = classify_question(sentence)\n    print(f"Sentence: {sentence}")\n    print(f"Predicted Label: {predicted_label}\n")'